In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


## Import library

In [ ]:
!pip install torch
!pip install transformers
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from sklearn.metrics import (accuracy_score, 
                             precision_recall_curve,
                             f1_score,
                             auc)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

## Load data & tokenization

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/preprocessing_train.csv", encoding='cp949')

In [ ]:
dev = pd.read_csv("/content/drive/MyDrive/preprocessing_dev.csv", encoding='cp949')
test = pd.read_csv("/content/drive/MyDrive/preprocessing_test.csv", encoding='cp949')

In [ ]:
train

,id,label,split,project,noncomment_code
0,libtiff_40a7486f8a551080b937ee325a5ec8c9527627...,0,train,libtiff,'static void TIFFWriteOvrRow( TIFFOvrCache * p...
1,libtiff_402be996607a5653134f1fc4b748989593de02...,0,train,libtiff,"'static int\ncvt_by_tile( TIFF *in, TIFF *out ..."
2,libtiff_864eb7cdda6f1722542f0b7cd115a984c52c3c...,0,train,libtiff,"'uint32 TIFF_WriteOverview( TIFF *hTIFF, uint3..."
3,libtiff_0e7f946d2aad122ea6fa4fff8bc120eba7b59a...,0,train,libtiff,'static void TIFFWriteOvrRow( TIFFOvrCache * p...
4,libtiff_22f83ba780fa385afa4d0441de5ecc3bab1d6c...,0,train,libtiff,'static int\ninitImage(void)\n{\n uint3...
...,...,...,...,...,...
1049928,ffmpeg_a4974a2ff0501ec333b624e32318a9c43a739bce_0,0,train,ffmpeg,"""static int get_pix_fmt_score(enum AVPixelForm..."
1049929,ffmpeg_905341030c7f92de10c132bd760a2ff34bb6696f_0,0,train,ffmpeg,"'static int get_sindex(AVFormatContext *s, int..."
1049930,ffmpeg_1c83c0d902ed823515aa27a2366a54d1ff56edc1_0,0,train,ffmpeg,'static void start_children(FFServerStream *fe...
1049931,ffmpeg_538c77e00d0b41cd7ba7c9128fddb5212891f1ed_0,0,train,ffmpeg,'static void assert_codec_experimental(AVCodec...


In [ ]:
train.isnull().sum()

id                   0
label                0
split                0
project              0
noncomment_code    514
dtype: int64

In [ ]:
train = train.dropna(axis=0)

In [ ]:
train = train.copy()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")

In [ ]:
MAX_LEN = 512
def tokenized(examples):
  return tokenizer(examples['noncomment_code'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
train.to_csv("preprocess.csv")

In [ ]:
dataset = load_dataset("csv",data_files="preprocess.csv")['train']

Computing checksums: 100%|##########| 1/1 [00:05<00:00,  5.98s/it]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-3fc2e2f5c5ef4e5d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'label', 'split', 'project', 'noncomment_code'],
    num_rows: 1049419
})

In [ ]:
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','id','split','project','noncomment_code'],batched=True) #12분

  0%|          | 0/1050 [00:00<?, ?ba/s]

In [ ]:
encoded_dataset=encoded_dataset.rename_column(original_column_name='label',new_column_name='labels')

In [ ]:
encoded_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1049419
})

In [ ]:
dev.isnull().sum()

id                  0
label               0
split               0
project             0
noncomment_code    57
dtype: int64

In [ ]:
dev = dev.dropna(axis=0)

In [ ]:
dev = dev.copy()

In [ ]:
# object -> str
#dev['noncomment_code'] = dev['noncomment_code'].astype(str)

In [ ]:
dev.to_csv("preprocess_dev.csv")

In [ ]:
dataset = load_dataset("csv",data_files="preprocess_dev.csv")['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-631306ac1620ee0e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

Dataset({
    features: ['Unnamed: 0', 'id', 'label', 'split', 'project', 'noncomment_code'],
    num_rows: 133061
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")

MAX_LEN = 512
def tokenized(examples):
  return tokenizer(examples['noncomment_code'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [ ]:
encoded_dataset_dev = dataset.map(tokenized,remove_columns=['Unnamed: 0','id','split','project','noncomment_code'],batched=True) #1분

  0%|          | 0/134 [00:00<?, ?ba/s]

In [ ]:
encoded_dataset_dev=encoded_dataset_dev.rename_column(original_column_name='label',new_column_name='labels')

In [ ]:
encoded_dataset_dev

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 133061
})

## train - finetuning

In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
DEVICE

device(type='cuda')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("neulab/codebert-c", num_labels=2)

Some weights of the model checkpoint at neulab/codebert-c were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neulab/codebert-c and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You

In [ ]:
model.to("cuda")
model.train()
for param in model.roberta.parameters():
    param.requires_grad = False

print(f"num params:", model.num_parameters())
print(f"num trainable params:", model.num_parameters(only_trainable=True))

num params: 124647170
num trainable params: 592130


In [ ]:
batch_size = 16
epoch_num = 1
learning_rate = 2e-5

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

'metric = load_metric("f1")\n\ndef compute_metrics(eval_pred):\n  predictions, labels = eval_pred\n  predictions = np.argmax(predictions, axis=1)\n  return metric.compute(predictions=predictions, references=labels)\n\n\ndef compute_metrics(pred):\n  labels = pred.label_ids\n  preds = pred.predictions.argmax(-1)\n\n  label_indices = list(range(3))\n  f1 = f1_score(labels, preds, average="macro", labels=label_indices) * 100.0\n  return {\'micro f1 score\': f1}\n  '

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8, 
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
args = TrainingArguments(save_strategy="steps",evaluation_strategy="steps",logging_strategy="steps", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=32,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model="f1",load_best_model_at_end=True,
                         logging_steps=300, eval_steps = 10000, save_steps=10000,
                         output_dir = '/content/drive/MyDrive/산학/log_codebert_c', logging_dir = '/content/drive/MyDrive/산학/log_codebert_c'
                         )

In [ ]:
trainer = Trainer(model,args,train_dataset=encoded_dataset,eval_dataset=encoded_dataset_dev,
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)

In [ ]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trainer.train() 

***** Running training *****
  Num examples = 1049419
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 65589
  Number of trainable parameters = 592130
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Auroc
10000,0.068500,0.079956,0.985052,0.496235,0.492526,0.500000,0.500000
20000,0.072600,0.080555,0.985052,0.496235,0.492526,0.500000,0.500000
30000,0.076000,0.082343,0.985052,0.496235,0.492526,0.500000,0.500000
40000,0.089000,0.081061,0.985052,0.496235,0.492526,0.500000,0.500000
50000,0.067400,0.079133,0.985052,0.496235,0.492526,0.500000,0.500000
60000,0.073800,0.080242,0.985052,0.496235,0.492526,0.500000,0.500000


***** Running Evaluation *****
  Num examples = 133061
  Batch size = 32
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/MyDrive/산학/log_codebert_c/checkpoint-10000
Configuration saved in /content/drive/MyDrive/산학/log_codebert_c/checkpoint-10000/config.json
Model weights saved in /content/drive/MyDrive/산학/log_codebert_c/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/산학/log_codebert_c/checkpoint-10000/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/산학/log_codebert_c/checkpoint-10000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 133061
  Batch size = 32
/usr/local/lib/python3.8/dist-packages/sklea

TrainOutput(global_step=65589, training_loss=0.07496445817552913, metrics={'train_runtime': 15199.2499, 'train_samples_per_second': 69.044, 'train_steps_per_second': 4.315, 'total_flos': 2.7611374060483584e+17, 'train_loss': 0.07496445817552913, 'epoch': 1.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/산학/codebert_c.pt')

Saving model checkpoint to /content/drive/MyDrive/산학/codebert_c.pt
Configuration saved in /content/drive/MyDrive/산학/codebert_c.pt/config.json
Model weights saved in /content/drive/MyDrive/산학/codebert_c.pt/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/산학/codebert_c.pt/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/산학/codebert_c.pt/special_tokens_map.json


## predict

In [ ]:
test = test.dropna(axis=0)
test.reset_index(drop=True, inplace=True)
test.to_csv("preprocess_test.csv")
dataset = load_dataset("csv",data_files="preprocess_test.csv")
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c892f9182f1a8511/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'label', 'split', 'project', 'noncomment_code'],
        num_rows: 131117
    })
})

In [ ]:
dataset = dataset['train']
tokenizer = AutoTokenizer.from_pretrained("neulab/codebert-c")
encoded_dataset_test = dataset.map(tokenized,remove_columns=['Unnamed: 0','id','split','project','noncomment_code'],batched=True) #1분
encoded_dataset_test = encoded_dataset_test.rename_column(original_column_name='label',new_column_name='labels')

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--neulab--codebert-c/snapshots/9ef09fbefc40181edc8c5ec703da9897cb373401/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--neulab--codebert-c/snapshots/9ef09fbefc40181edc8c5ec703da9897cb373401/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--neulab--codebert-c/snapshots/9ef09fbefc40181edc8c5ec703da9897cb373401/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--neulab--codebert-c/snapshots/9ef09fbefc40181edc8c5ec703da9897cb373401/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--neulab--codebert-c/snapshots/9ef09fbefc40181edc8c5ec703da9897cb373401/tokenizer_config.json


  0%|          | 0/132 [00:00<?, ?ba/s]

In [ ]:
predictions = trainer.predict(encoded_dataset_test)
predictions

***** Running Prediction *****
  Num examples = 131117
  Batch size = 32


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PredictionOutput(predictions=array([[ 2.6218438, -2.2046964],
       [ 2.6873987, -2.244136 ],
       [ 2.6103673, -2.247733 ],
       ...,
       [ 2.8214033, -2.480319 ],
       [ 2.7736983, -2.4150426],
       [ 2.7736983, -2.4150426]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.0756084993481636, 'test_accuracy': 0.9858141964810055, 'test_f1': 0.4964282147987126, 'test_precision': 0.4929070982405028, 'test_recall': 0.5, 'test_auroc': 0.5, 'test_runtime': 996.5952, 'test_samples_per_second': 131.565, 'test_steps_per_second': 4.112})

In [ ]:
preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
np.unique(preds, return_counts = True)

(array([0]), array([131117]))